In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/train.csv
/kaggle/input/bike-sharing-demand/test.csv


In [13]:
training= pd.read_csv('../input/bike-sharing-demand/train.csv')
test=pd.read_csv('../input/bike-sharing-demand/test.csv')
training.head(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [14]:
training.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

In [15]:
test.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
dtype: int64

In [16]:
new_windspeed_train=training['windspeed'].replace([0],[12.79])
new_windspeed_test=test['windspeed'].replace([0],[12.63])

In [17]:
training=training.drop(['casual','registered','datetime','windspeed'],axis=1)
training['windspeed']=new_windspeed_train

In [18]:
test=test.drop(['windspeed'],axis=1)
test['windspeed']=new_windspeed_test

In [19]:
new_windspeed_train.describe()

count    10886.000000
mean        14.342044
std          6.672212
min          6.003200
25%          8.998100
50%         12.998000
75%         16.997900
max         56.996900
Name: windspeed, dtype: float64

In [20]:
training['windspeed'].describe()

count    10886.000000
mean        14.342044
std          6.672212
min          6.003200
25%          8.998100
50%         12.998000
75%         16.997900
max         56.996900
Name: windspeed, dtype: float64

In [21]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test = tts(training.drop(['count'],axis=1),training['count'],test_size = 0.2,random_state = 1)
print(X_train.shape, type(X_train))
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8708, 8) <class 'pandas.core.frame.DataFrame'>
(2178, 8)
(8708,)
(2178,)


In [22]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error as rmlse
lg=LinearRegression()
lg.fit(X_train,y_train)
ans_df = pd.DataFrame(columns=['datetime','count'])
ans_df['datetime']=test['datetime']
ans_df['count']=lg.predict(test.drop(['datetime'],axis=1))
ans_df.to_csv('solution_lg.csv',index=False)
y_pred = lg.predict(X_test)
#rmlse(y_test, y_pred)

In [23]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor()
dtr.fit(X_train,y_train)
ans_df = pd.DataFrame(columns=['datetime','count'])
ans_df['datetime']=test['datetime']
ans_df['count']=dtr.predict(test.drop(['datetime'],axis=1))
ans_df.to_csv('solution_dtr.csv',index=False)
y_pred = dtr.predict(X_test)
rmlse(y_test, y_pred)

2.25720950177432

In [24]:
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor()
rfr.fit(X_train,y_train)
ans_df = pd.DataFrame(columns=['datetime','count'])
ans_df['datetime']=test['datetime']
ans_df['count']=rfr.predict(test.drop(['datetime'],axis=1))
ans_df.to_csv('solution_rfr.csv',index=False)
y_pred = rfr.predict(X_test)
rmlse(y_test, y_pred)

1.5475636281956946

In [25]:
from sklearn.ensemble import GradientBoostingRegressor
gbr=GradientBoostingRegressor()
gbr.fit(X_train,y_train)
ans_df = pd.DataFrame(columns=['datetime','count'])
ans_df['datetime']=test['datetime']
ans_df['count']=gbr.predict(test.drop(['datetime'],axis=1))
ans_df.to_csv('solution_gbr.csv',index=False)
y_pred =gbr.predict(X_test)
rmlse(y_test, y_pred)

1.6946831807540006